In [30]:
import pandas as pd
import numpy as np

class StudentDataPipeline:
    def __init__(self, window=10):
        # We initialize our pipeline and define which columns belong to which category
        self.window = window
        self.id_columns = ["student_id"]
        self.numeric_columns = ["attendance", "project_score", "subject_score"]
        self.categorical_columns = ["income_student", "track"]
        self.percent_columns = ["attendance"]
        self.allowed_categories = {"income_student": ["Yes", "No"]}

    # ---------------------------
    # LOAD & MERGE
    # ---------------------------
    def load_excel(self, path):
        # We detect the correct Excel engine and load all sheets
        engine = "openpyxl" if path.lower().endswith(".xlsx") else "xlrd"
        sheets = pd.read_excel(path, sheet_name=None, engine=engine, dtype=object)

        frames = []
        # We iterate over each sheet, attach its sheet name as a track, and collect the dataframes
        for name, df in sheets.items():
            if df is None or df.empty:
                continue
            df = df.copy()
            df["track"] = name
            frames.append(df)

        # We merge all sheets into a single dataframe
        if not frames:
            return pd.DataFrame()
        merged = pd.concat(frames, ignore_index=True, sort=False)
        return merged

    # ---------------------------
    # COLUMN NORMALIZATION
    # ---------------------------
    def normalize_columns(self, df):
        # We standardize column names and rename known variants to a unified schema
        df = df.copy()
        df.columns = [c.strip().lower().replace(" ", "_") if isinstance(c, str) else c for c in df.columns]

        rename_map = {
            "student_id": "student_id",
            "attendance_%": "attendance",
            "attendance": "attendance",
            "project": "project_score",
            "project_score": "project_score",
            "subject": "subject_score",
            "subject_score": "subject_score",
            "income_student": "income_student",
            "helped": "helped",
            "helped?": "helped",
            "income": "income",
        }
        df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns}, inplace=True)

        # We remove columns that should not be kept
        for col in ("helped", "income"):
            if col in df.columns:
                df.drop(columns=[col], inplace=True)
        return df

    # ---------------------------
    # CLEANING & VALUE NORMALIZATION
    # ---------------------------
    def normalize_values(self, df):
        # We trim whitespace and prepare string values
        df = df.copy()
        for c in df.columns:
            if df[c].dtype == object:
                df[c] = df[c].apply(lambda x: x.strip() if isinstance(x, str) else x)

        # We replace known missing-value tokens with NaN
        missing_tokens = {"waive", "waived", "waiver", "n/a", "na", "none", "null", "-", "--", ""}
        for c in df.columns:
            df[c] = df[c].apply(lambda x: np.nan if isinstance(x, str) and x.strip().lower() in missing_tokens else x)

        # We normalize categorical values for income_student
        if "income_student" in df.columns:
            s = df["income_student"].astype("string").str.strip().str.lower()
            mapping = {
                "yes": "Yes", "y": "Yes", "true": "Yes", "t": "Yes", "1": "Yes",
                "no": "No", "n": "No", "false": "No", "f": "No", "0": "No",
            }
            df["income_student"] = s.map(mapping)
            df["income_student"] = df["income_student"].where(df["income_student"].notna(), np.nan)
        return df

    # ---------------------------
    # TYPE CONVERSIONS
    # ---------------------------
    def convert_dtypes(self, df):
        # We convert percentage-like columns into numeric percentages
        df = df.copy()
        for col in self.percent_columns:
            if col in df.columns:
                s = df[col].astype(str)
                has_pct = s.str.contains("%", na=False)
                s = s.str.replace("%", "", regex=False)
                s = pd.to_numeric(s, errors="coerce")
                s = pd.Series(np.where(has_pct, s, np.where(s <= 1, s * 100, s)), index=df.index)
                df[col] = s

        # We convert numeric columns to floats
        for col in self.numeric_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce")

        # We ensure income_student stays as string
        if "income_student" in df.columns:
            df["income_student"] = df["income_student"].astype("string")
        return df

    # ---------------------------
    # MISSING VALUE HANDLING
    # ---------------------------
    def impute_missing(self, df):
        # We fill missing numeric and categorical values using rolling statistics
        df = df.copy()
        df = df.sort_index()

        # We apply rolling mean imputation for numeric columns
        for col in self.numeric_columns:
            if col in df.columns:
                stats = df[col].shift(1).rolling(window=self.window, min_periods=1).mean()
                df[col] = df[col].fillna(stats)
                if df[col].isna().any():
                    df[col] = df[col].fillna(df[col].mean())

        # We apply rolling mode imputation for categorical income_student
        if "income_student" in df.columns:
            rolled_mode = (
                df["income_student"].shift(1)
                .rolling(window=self.window, min_periods=1)
                .apply(lambda x: pd.Series(x).mode().iloc[0] if not pd.Series(x).mode().empty else np.nan)
            )
            df["income_student"] = df["income_student"].fillna(rolled_mode)
            if df["income_student"].isna().any():
                mode = df["income_student"].mode(dropna=True)
                if not mode.empty:
                    df["income_student"] = df["income_student"].fillna(mode.iloc[0])
        return df

    # ---------------------------
    # SUMMARY ROW (APPEND AT END)
    # ---------------------------
    def append_overall_summary_row(self, df: pd.DataFrame) -> pd.DataFrame:
        # We compute summary statistics and append a synthetic row at the end
        num_cols = [c for c in ["attendance", "subject_score", "project_score"] if c in df.columns]
        n_students = df["student_id"].value_counts().count() if "student_id" in df.columns else len(df)

        if "student_count" not in df.columns:
            df = df.copy()
            df["student_count"] = np.nan

        row = {c: np.nan for c in df.columns}
        if "track" in df.columns:
            row["track"] = "__OVERALL__"
        for c in num_cols:
            row[c] = df[c].mean(skipna=True)
        row["student_count"] = n_students

        return pd.concat([df, pd.DataFrame([row], columns=df.columns)], ignore_index=True)

    # ---------------------------
    # VALIDATION & EXPORT
    # ---------------------------
    def validate(self, df):
        # We check for invalid ranges and unexpected categorical values
        report = {}
        for col in ["attendance", "project_score", "subject_score"]:
            if col in df.columns:
                invalid = df[(df[col].notna()) & ((df[col] < 0) | (df[col] > 100))]
                report[f"{col}_out_of_range"] = len(invalid)

        if "income_student" in df.columns:
            bad = df["income_student"].dropna().astype("string")
            report["income_student_invalid"] = int((~bad.isin(self.allowed_categories["income_student"])).sum())

        report["remaining_nulls_per_column"] = {
            col: int(df[col].isna().sum()) for col in df.columns if df[col].isna().any()
        }
        return report

    def export_csv(self, df, path):
        # We export the cleaned dataframe to CSV
        df.to_csv(path, index=False)

    # ---------------------------
    # FULL PIPELINE RUNNER
    # ---------------------------
    def run(self, excel_path, output_csv):
        # We orchestrate all processing steps of the pipeline
        df = self.load_excel(excel_path)
        df = self.normalize_columns(df)
        df = self.normalize_values(df)
        df = self.convert_dtypes(df)
        df = self.impute_missing(df)

        report = self.validate(df)
        self.export_csv(df, output_csv)

        # We print results to confirm completion
        print("Pipeline complete.")
        print("Validation report:", report)
        print(f"Exported to {output_csv}")


# Example usage:
if __name__ == "__main__":
    pipeline = StudentDataPipeline(window=10)
    pipeline.run("students.xlsx", "merged.csv")

Pipeline complete.
Validation report: {'remaining_nulls_per_column': {'studentid': 1, 'math': 9, 'english': 6, 'science': 6, 'history': 4, 'attendance_(%)': 13, 'incomestudent': 1}}
Exported to merged.csv


In [31]:
import pandas as pd
import numpy as np

# -------------------------------------------------------
# UNIVERSAL COLUMN FINDER (ROBUST FOR ANY DATASET)
# -------------------------------------------------------

def find_column(df, keywords):
    """
    We try to find a column whose name contains any of the given keywords.
    Consequently, we normalize column names to make matching easier.
    """
    normalized = {
        col: col.lower().replace(" ", "").replace("_", "")
        for col in df.columns
    }

    # Here, we iterate through each keyword and, subsequently, each normalized column
    for key in keywords:
        key_norm = key.lower().replace(" ", "").replace("_", "")
        for col, norm in normalized.items():
            # If the keyword appears in the column name, we immediately return it
            if key_norm in norm:
                return col
    return None


# -------------------------------------------------------
# DETECT METRIC COLUMNS (math, english, history, project…)
# -------------------------------------------------------
def get_metric_numeric_columns(df):
    # Firstly, we collect all numeric columns from the dataframe
    all_numeric = df.select_dtypes(include=[np.number]).columns.tolist()

    # Then, we define keywords that should help us identify academic scores
    keep_keywords = [
        "attend", "score", "grade", "note", "math", "english",
        "science", "history", "project", "exam", "test", "mark"
    ]
    drop_keywords = ["id", "code", "phone", "num", "year", "age"]

    metrics = []
    # We check each numeric column and, therefore, decide whether it is an academic metric
    for col in all_numeric:
        name = col.lower()
        if any(k in name for k in keep_keywords) and not any(b in name for b in drop_keywords):
            metrics.append(col)

    return metrics


# -------------------------------------------------------
# PASS RATE HELPER
# -------------------------------------------------------
def compute_pass_rate(series):
    # Here, we convert each value to a lowercase string so that comparisons become consistent
    s = series.astype(str).str.lower().str.strip()

    # Next, we define words that represent "yes" or "no" in various forms
    yes = {"y", "yes", "1", "true", "t", "pass", "passed"}
    no = {"n", "no", "0", "false", "f", "fail", "failed"}

    # We normalize the pass column into Yes/No values, accordingly
    normalized = s.apply(lambda x: "Yes" if x in yes else ("No" if x in no else np.nan))

    # Finally, we compute the percentage of "Yes" values
    return (normalized == "Yes").mean() * 100


# -------------------------------------------------------
# MAIN STATS REPORT (TRACK + COHORT)
# -------------------------------------------------------
def compute_track_stats(cleaned_csv, stats_output_excel):
    # Initially, we load the cleaned dataset
    df = pd.read_csv(cleaned_csv)

    # ---------------------------------------------------
    # AUTO-DETECT IMPORTANT COLUMNS
    # ---------------------------------------------------
    # Here, we attempt to automatically determine essential column names
    track_col = find_column(df, ["track"])
    cohort_col = find_column(df, ["cohort", "cohorte", "promo", "generation", "year"])
    pass_col = find_column(df, ["pass", "passed", "result", "status"])
    attendance_col = find_column(df, ["attendance", "attend", "presence"])
    project_col = find_column(df, ["project", "projectscore", "proj"])

    if track_col is None:
        raise ValueError("ERROR: Track column not found.")

    # We normalize column names by renaming them once detected
    df.rename(columns={track_col: "track"}, inplace=True)
    if cohort_col: df.rename(columns={cohort_col: "cohort"}, inplace=True)
    if pass_col: df.rename(columns={pass_col: "pass_col"}, inplace=True)
    if attendance_col: df.rename(columns={attendance_col: "attendance"}, inplace=True)
    if project_col: df.rename(columns={project_col: "projectscore"}, inplace=True)

    # We convert attendance and projectscore to numeric, because calculations require numbers
    if "attendance" in df.columns:
        df["attendance"] = pd.to_numeric(df["attendance"], errors="coerce")
    if "projectscore" in df.columns:
        df["projectscore"] = pd.to_numeric(df["projectscore"], errors="coerce")

    # Now, we detect all academic metric columns dynamically
    metric_cols = get_metric_numeric_columns(df)

    # ---------------------------------------------------
    # TRACK-LEVEL STATS
    # ---------------------------------------------------
    # We compute the number of students per track
    track_count = df.groupby("track").size().rename("student_count")

    # Next, we compute the mean of each metric by track
    track_mean = df.groupby("track")[metric_cols].mean()

    # Additionally, we compute the pass rate per track when possible
    if "pass_col" in df.columns:
        track_pass = df.groupby("track")["pass_col"].apply(compute_pass_rate).rename("pass_rate")
    else:
        track_pass = pd.Series(np.nan, index=track_count.index, name="pass_rate")

    # Then, we compute the correlation between attendance and project score for each track
    corr_series = pd.Series(index=track_count.index, dtype=float, name="corr_attendance_project")
    if "attendance" in df.columns and "projectscore" in df.columns:
        for t, g in df.groupby("track"):
            corr_series.loc[t] = g["attendance"].corr(g["projectscore"])
    else:
        corr_series[:] = np.nan

    # Consequently, we combine all computed metrics into a single dataframe
    track_stats = pd.concat([track_count, track_mean, track_pass, corr_series], axis=1).reset_index()

    # ---------------------------------------------------
    # ADD OVERALL ROW TO TRACK SHEET
    # ---------------------------------------------------
    # We add a summary row that synthesizes overall statistics
    overall_track = {"track": "__OVERALL__", "student_count": track_count.sum()}

    for col in track_stats.columns:
        if col not in ["track", "student_count"]:
            overall_track[col] = ""

    track_stats = pd.concat([track_stats, pd.DataFrame([overall_track])], ignore_index=True)

    # ---------------------------------------------------
    # COHORT-LEVEL STATS (WITH OVERALL)
    # ---------------------------------------------------
    if "cohort" in df.columns:
        # As before, we compute cohort-level counts and metrics
        cohort_count = df.groupby("cohort").size().rename("student_count")
        cohort_mean = df.groupby("cohort")[metric_cols].mean()

        if "pass_col" in df.columns:
            cohort_pass = df.groupby("cohort")["pass_col"].apply(compute_pass_rate).rename("pass_rate")
        else:
            cohort_pass = pd.Series(np.nan, index=cohort_count.index, name="pass_rate")

        cohort_stats = pd.concat([cohort_count, cohort_mean, cohort_pass], axis=1).reset_index()

        # Then, we add an overall summary row for cohorts
        overall_cohort = {"cohort": "__OVERALL__", "student_count": cohort_count.sum()}

        for col in cohort_stats.columns:
            if col not in ["cohort", "student_count"]:
                overall_cohort[col] = ""

        cohort_stats = pd.concat([cohort_stats, pd.DataFrame([overall_cohort])], ignore_index=True)

    else:
        # If no cohort column exists, we clearly indicate it
        cohort_stats = pd.DataFrame({"cohort": ["No cohort column found"], "student_count": [""]})

    # ---------------------------------------------------
    # SAVE EXCEL WITH TWO SHEETS
    # ---------------------------------------------------
    # Finally, we save our results in an Excel file with two separate sheets
    with pd.ExcelWriter(stats_output_excel, engine="openpyxl") as writer:
        track_stats.to_excel(writer, sheet_name="By_Track", index=False)
        cohort_stats.to_excel(writer, sheet_name="By_Cohort", index=False)

    # We also print previews to help us quickly verify the results
    print("\n=== Stats Report Generated ===")
    print("Saved to:", stats_output_excel)
    print("\nTrack sheet preview:")
    print(track_stats.tail())
    print("\nCohort sheet preview:")
    print(cohort_stats.tail())


# -------------------------------------------------------
# RUN
# -------------------------------------------------------
if __name__ == "__main__":
    # We simply call the main function when running the script
    compute_track_stats("merged.csv", "stats_report.xlsx")



=== Stats Report Generated ===
Saved to: stats_report.xlsx

Track sheet preview:
         track  student_count       math    english    science    history  \
0           BM            200   64.45736  64.968342  65.989447  64.824121   
1         Data            200  72.308081  71.323232  72.532995  72.737374   
2      Finance            200  62.434694  64.948731  66.466667  63.536683   
3  __OVERALL__            600                                               

  attendance projectscore pass_rate corr_attendance_project  
0  78.997462       71.398      91.5                -0.05283  
1  84.137436         72.2      90.0               -0.068687  
2  79.963077      69.2345      95.0               -0.047074  
3                                                            

Cohort sheet preview:
        cohort  student_count       math    english    science    history  \
0        25-26            198  66.655897  68.252792  69.618653  67.538071   
1        26-27            194  67.153684  65.

In [38]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# ----------------------------------------------
# UNIVERSAL COLUMN FINDER (reuse this everywhere)
# ----------------------------------------------
def find_column(df, keywords):
    # We first normalize all column names so that matching becomes easier and, consequently, more reliable
    normalized = {
        col: col.lower().replace(" ", "").replace("_", "")
        for col in df.columns
    }
    # Then, we iterate through each keyword and compare with each normalized column name
    for key in keywords:
        key_norm = key.lower().replace(" ", "").replace("_", "")
        for col, norm in normalized.items():
            # If a keyword is contained in a column name, we immediately return it
            if key_norm in norm:
                return col
    # If nothing matches, we return None because we want to avoid false detections
    return None


# -------------------------------------------------------
# SAFE HISTORY DISTRIBUTION PLOTS (per track)
# -------------------------------------------------------
def plot_history_per_track(cleaned_csv):
    # First, we load the cleaned CSV so we can start analyzing the data
    df = pd.read_csv(cleaned_csv)

    # 1) Find the history column safely
    # Here, we attempt to locate any column related to history by checking several keyword variants
    history_col = find_column(df, ["history", "hist", "histoire", "socialscience"])
    if history_col is None:
        # If no such column is found, we stop the process because we cannot plot what does not exist
        raise ValueError("Could not find any history-like column in dataset.")

    # 2) Convert to numeric (fix WAIVE, strings, etc.)
    # We then attempt to convert the identified column to numeric, since history scores must be numbers
    df[history_col] = pd.to_numeric(df[history_col], errors="coerce")

    # We remove rows where the history score is missing because they cannot be plotted meaningfully
    df = df.dropna(subset=[history_col])

    # 3) Find track column safely
    # Next, we locate the track column so we can produce one histogram per track
    track_col = find_column(df, ["track"])
    if track_col is None:
        raise ValueError("Track column missing.")

    # 4) Plot one histogram per track
    # We extract the list of unique track names so we can iterate through them
    tracks = df[track_col].unique()

    for t in tracks:
        # For each track, we isolate the subset of students belonging to it
        subset = df[df[track_col] == t]

        # If the subset is empty (which rarely happens), we skip it entirely
        if subset.empty:
            continue

        # We create a large figure so the histogram looks clear
        plt.figure(figsize=(12, 6))

        # Here, we plot the distribution using seaborn's histplot, adding a KDE to better visualize the shape
        sns.histplot(
            subset[history_col],
            bins=20,
            kde=True,
            color="darkorchid",
            alpha=0.7
        )

        # We add a title and axis labels to ensure the plot is understandable
        plt.title(f"Distribution of {history_col} — {t} Track")
        plt.xlabel(history_col)
        plt.ylabel("Count")
        plt.grid(alpha=0.3)
        plt.tight_layout()

        # We save the plot so we can inspect it later
        plt.savefig(f"{t}_history.png")

        # We close the figure to avoid excessive memory usage
        plt.close()

        # We show the plot directly, in case we're running this interactively
        plt.show()

    # Finally we print which column was identified as history to give the user quick feedback
    print("History column detected:", history_col)

In [33]:
plot_history_per_track("merged.csv")

History column detected: history


In [34]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# -------------------------------------------------------
# UNIVERSAL COLUMN FINDER (reuse this)
# -------------------------------------------------------
# We add detailed comments written as students reflecting on our reasoning.
def find_column(df, keywords):
    # Here we begin by normalizing every column name so that comparisons become simpler and, consequently, more reliable
    normalized = {
        col: col.lower().replace(" ", "").replace("_", "")
        for col in df.columns
    }

    # Next, we iterate through each keyword, because we want to check whether any of them appears in a column name
    for key in keywords:
        key_norm = key.lower().replace(" ", "").replace("_", "")
        for col, norm in normalized.items():
            # If the keyword is included in a column, we return that column immediately
            if key_norm in norm:
                return col
    # If nothing matches, we logically return None
    return None


# -------------------------------------------------------
# Detect academic metric numerical columns
# -------------------------------------------------------
def get_metric_numeric_columns(df):
    # First, we extract all numeric columns because only these can represent academic scores
    all_numeric = df.select_dtypes(include=[np.number]).columns.tolist()

    # Then, we define keywords that usually indicate academic performance
    keep_keywords = [
        "attend", "score", "grade", "note", "math", "english",
        "science", "history", "project", "exam", "test", "mark"
    ]

    # Conversely, we exclude numeric columns that clearly do not represent academic scores
    drop_keywords = ["id", "num", "code", "year", "age"]

    metrics = []
    # We evaluate every numeric column to determine whether it should be kept
    for col in all_numeric:
        name = col.lower()
        if any(k in name for k in keep_keywords) and not any(b in name for b in drop_keywords):
            metrics.append(col)

    return metrics


# -------------------------------------------------------
# Income vs Non-Income Comparison (Grouped Bar Plot)
# -------------------------------------------------------
def plot_income_comparison(cleaned_csv):
    # We load the cleaned data so that we can perform our comparison
    df = pd.read_csv(cleaned_csv)

    # 1) Detect the income column automatically
    # Here, we try to locate any column related to financial support or scholarships
    income_col = find_column(df, ["income", "support", "aid", "help", "bourse", "scholar"])
    if income_col is None:
        raise ValueError("Could not find income/student-support column in dataset.")

    # We convert all values into a consistent Yes/No format so our comparisons are meaningful
    df[income_col] = df[income_col].astype(str).str.lower().str.strip()
    df[income_col] = df[income_col].map({
        "yes": "Yes", "y": "Yes", "1": "Yes", "true": "Yes", "vrai": "Yes",
        "no": "No", "n": "No", "0": "No", "false": "No", "faux": "No"
    })

    # 2) Identify academic metric columns
    # Now we determine which numeric columns represent academic results
    metric_cols = get_metric_numeric_columns(df)

    # Then, we make sure these values are numeric so the averages we compute later are correct
    for col in metric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # 3) Compute the average per income group
    # Here we group students by whether they receive financial support and calculate the mean of each metric
    summary = df.groupby(income_col)[metric_cols].mean()

    # We transpose so that subjects appear on rows, which consequently makes the bar plot more readable
    summary = summary.T

    # 4) Plot grouped bar chart
    # We create a bar chart comparing performance between students with and without income support
    plt.figure(figsize=(14, 7))
    summary.plot(kind="bar", color=["darkorchid", "indigo"], figsize=(12, 6))

    # We label our plot carefully so viewers immediately understand what is being shown
    plt.title("Academic Performance — Income Support vs Non-Support")
    plt.xlabel("Subjects")
    plt.ylabel("Average Score")
    plt.xticks(rotation=45)
    plt.grid(axis="y", alpha=0.3)
    plt.legend(title="Income Support")
    plt.tight_layout()

    # We save the figure so we can use it later
    plt.savefig("income_support_vs_non.png")

    # We close and then show the plot to avoid clutter
    plt.close()
    plt.show()

    # Finally, we print a quick summary so we can verify the correct columns were detected
    print("Income column detected:", income_col)
    print("Metrics used:", metric_cols)
    print("Summary:")
    print(summary)

In [35]:
plot_income_comparison("merged.csv")

<Figure size 1400x700 with 0 Axes>

Income column detected: incomestudent
Metrics used: ['math', 'english', 'science', 'history', 'attendance_(%)', 'projectscore']
Summary:
incomestudent          No        Yes
math            67.678386  64.477778
english         67.474857  66.600412
science         68.796275  67.673361
history         67.461254  66.386066
attendance_(%)  81.622899  80.201245
projectscore    70.918697  71.028862


In [36]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# ----------------------------------------------
# UNIVERSAL COLUMN FINDER
# ----------------------------------------------
# We write comments as students thinking through each step, using explicit reasoning and linking words.
def find_column(df, keywords):
    # First, we normalize all column names so that, consequently, comparisons become easier
    normalized = {
        col: col.lower().replace(" ", "").replace("_", "")
        for col in df.columns
    }
    # Next, we iterate through every keyword and compare with the normalized names
    for key in keywords:
        key_norm = key.lower().replace(" ", "").replace("_", "")
        for col, norm in normalized.items():
            # If the keyword is found inside a column name, we immediately select it
            if key_norm in norm:
                return col
    # If we reach this point, then no matching column was found
    return None


# -------------------------------------------------------
# COMPARAISON DES MOYENNES DE MATH PAR TRACK
# -------------------------------------------------------
def plot_math_by_track(cleaned_csv):
    # To begin with, we load the cleaned CSV file into a DataFrame
    df = pd.read_csv(cleaned_csv)

    # 1) Trouver la colonne Track automatiquement
    # Here, we try to identify which column corresponds to the track/class/section
    track_col = find_column(df, ["track", "classe", "section"])
    if track_col is None:
        raise ValueError("Could not find track column in dataset.")

    # 2) Trouver la colonne Math automatiquement
    # Likewise, we attempt to locate any column that likely represents math results
    math_col = find_column(df, ["math", "mathematique", "note_math", "note", "mat"])
    if math_col is None:
        raise ValueError("Could not find math column in dataset.")

    # 3) Convertir en numérique (safe)
    # Since math grades must be numeric for averaging, we safely convert them
    df[math_col] = pd.to_numeric(df[math_col], errors="coerce")

    # 4) Calculer les moyennes par track
    # Then, we compute the average math score per track; consequently, we obtain a clear comparison
    summary = df.groupby(track_col)[math_col].mean().sort_values(ascending=False)

    # 5) Plot
    # Now we create a bar plot to visualize differences in average math performance across tracks
    plt.figure(figsize=(12, 6))
    sns.barplot(
        x=summary.index,
        y=summary.values,
        palette="Purples",
    )

    # We add a title and axis labels so that the plot remains easy to interpret
    plt.title("Average Math Score per Track", fontsize=14)
    plt.xlabel("Track")
    plt.ylabel("Average Math Score")
    plt.grid(axis="y", alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Save
    # We save the figure in case we need to include it in a report later
    plt.savefig("math_by_track.png")
    plt.close()
    plt.show()

    # Finally, we print the detected columns and the summary table to verify everything makes sense
    print("Track column detected :", track_col)
    print("Math column detected :", math_col)
    print("\nSummary (math mean per track):")
    print(summary)


In [37]:
plot_math_by_track("merged.csv")

Track column detected : track
Math column detected : math

Summary (math mean per track):
track
Data       72.308081
BM         64.457360
Finance    62.434694
Name: math, dtype: float64


C:\Users\PC\AppData\Local\Temp\ipykernel_3156\1112421470.py:57: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
